# SI 618 Fall 2019 Homework 2 (100 points)

## Part 1 (75 points) 

The provided ‘movie_actors_data.txt’ file contains a JSON string on each line. For example, the first line is: 

>{"rating": 9.3, "genres": ["Crime", "Drama"], "rated": "R", "filming_locations": "Ashland, Ohio, USA", "language": ["English"], "title": "The Shawshank Redemption", "runtime": ["142 min"], "poster": "http://img3.douban.com/lpic/s1311361.jpg", "imdb_url": "http://www.imdb.com/title/tt0111161/", "writers": ["Stephen King", "Frank Darabont"], "imdb_id": "tt0111161", "directors": ["Frank Darabont"], "rating_count": 894012, "actors": ["Tim Robbins", "Morgan Freeman", "Bob Gunton", "William Sadler", "Clancy Brown", "Gil Bellows", "Mark Rolston", "James Whitmore", "Jeffrey DeMunn", "Larry Brandenburg", "Neil Giuntoli", "Brian Libby", "David Proval", "Joseph Ragno", "Jude Ciccolella"], "plot_simple": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.", "year": 1994, "country": ["USA"], "type": "M", "release_date": 19941014, "also_known_as": ["Die Verurteilten"]}

The fields we are interested in are imdb_id , title , rating, genres, actors, and year. You will parse the JSON strings, and load the data into three tables in SQLite, and then write SQL queries to retrieve the data specified. 

You will create three tables: 
* The “movie_genre” table, which has two columns: imdb_id and genre. A movie typically has multiple genres, and in this case, there should be one row for each genre. If some movie does not have any genre, ignore that movie.  
* The “movies” table, which has four columns: imdb_id, title, year, rating, country  
* The “movie_actor” table, which has two columns imdb_id and actor. A movie typically has  multiple actors, and in this case, there should be one row for each actor. 

**When you run your Python code, it should print out EXACTLY the output provided.**

#### Q1. (10 points) Parse input file to get needed data for the three tables and load them into appropriate Python data structure. 

In [53]:
import json
import sqlite3
data = []
with open('movie_actors_data.txt') as f:
    for line in f.readlines():
        data.append({k: json.loads(line)[k] for k in ['imdb_id','genres','title', 'year', 'rating', 'country','actors']})

#### Q2. (5 points) Create the movie_genre table and load data into it.

In [54]:
conn = sqlite3.connect('theoliao_si618_hw2.db')
c = conn.cursor()
c.execute('CREATE TABLE movie_genre (imdb_id text,genre text)')
rows_movie_genre = [(d['imdb_id'],g) for d in data for g in d['genres']]
c.executemany('INSERT INTO movie_genre VALUES (?,?)',rows_movie_genre)
conn.commit()

#### Q3. (5 points) Create the movies table and load data into it.

In [55]:
c.execute('CREATE TABLE movies (imdb_id text,title text,year integer,rating real,country text)')
rows_movies = [(d['imdb_id'],d['title'],d['year'],d['rating'],d['country'][0]) for d in data]
c.executemany('INSERT INTO movies VALUES (?,?,?,?,?)',rows_movies)
conn.commit()

#### Q4. (5 points) Create the movie_actor table and load data into it.

In [56]:
c.execute('CREATE TABLE movie_actor (imdb_id text,actor text)')
rows_movie_actor = [(d['imdb_id'],a) for d in data for a in d['actors']]
c.executemany('INSERT INTO movie_actor VALUES (?,?)',rows_movie_actor)
conn.commit()

#### Q5. (10 points) Write an SQL query to find top 10 genres with largest number of movies in that genre and print out the results.
```
Top 10 genres:Genre, Movies   
Drama,166 
Thriller,66  
Crime,60  
Adventure,56  
Mystery,42  
Comedy,41  
Action,40  
Romance,35  
Fantasy,30  
War,29
```

In [57]:
results = c.execute('SELECT genre, Count(*) FROM movie_genre GROUP BY genre ORDER BY Count(*) DESC LIMIT 10')
print("Top 10 genres:Genre, Movies") 
for row in results:
    print(row[0]+','+repr(row[1]))

Top 10 genres:Genre, Movies
Drama,166
Thriller,66
Crime,60
Adventure,56
Mystery,42
Comedy,41
Action,40
Romance,35
Fantasy,30
War,29


#### Q6. (10 points) Write an SQL query to find the number of movies broken down by year in chronological order.
```
Movies broken down by year:Year, Movies
1921, 1
1922, 1
1925, 1
1926, 1
1927, 1
1930, 1
1931, 2
1934, 1
1936, 1
1939, 3
1940, 3
1941, 2
1942, 1
1943, 1
1944, 2
1946, 3
1948, 3
1949, 1
1950, 4
1951, 2
1952, 2
1953, 2
1954, 5
1955, 1
1956, 1
1957, 6
1958, 2
1959, 5
1960, 2
1961, 2
1962, 4
1963, 2
1964, 2
1965, 1
1966, 3
1967, 2
1968, 3
1969, 2
1971, 1
1972, 2
1973, 3
1974, 2
1975, 5
1976, 3
1977, 2
1978, 1
1979, 5
1980, 3
1981, 2
1982, 3
1983, 1
1984, 3
1985, 2
1986, 3
1987, 3
1988, 5
1989, 1
1990, 1
1991, 3
1992, 2
1993, 4
1994, 5
1995, 8
1996, 2
1997, 3
1998, 5
1999, 5
2000, 5
2001, 5
2002, 5
2003, 9
2004, 5
2005, 3
2006, 4
2007, 7
2008, 7
2009, 7
2010, 7
2011, 6
2012, 8
```

In [58]:
results = c.execute('SELECT year, Count(*) FROM movies GROUP BY year ORDER BY year')
print("Movies broken down by year:Year, Movies") 
for row in results:
    print(repr(row[0])+', '+repr(row[1]))

Movies broken down by year:Year, Movies
1921, 1
1922, 1
1925, 1
1926, 1
1927, 1
1930, 1
1931, 2
1934, 1
1936, 1
1939, 3
1940, 3
1941, 2
1942, 1
1943, 1
1944, 2
1946, 3
1948, 3
1949, 1
1950, 4
1951, 2
1952, 2
1953, 2
1954, 5
1955, 1
1956, 1
1957, 6
1958, 2
1959, 5
1960, 2
1961, 2
1962, 4
1963, 2
1964, 2
1965, 1
1966, 3
1967, 2
1968, 3
1969, 2
1971, 1
1972, 2
1973, 3
1974, 2
1975, 5
1976, 3
1977, 2
1978, 1
1979, 5
1980, 3
1981, 2
1982, 3
1983, 1
1984, 3
1985, 2
1986, 3
1987, 3
1988, 5
1989, 1
1990, 1
1991, 3
1992, 2
1993, 4
1994, 5
1995, 8
1996, 2
1997, 3
1998, 5
1999, 5
2000, 5
2001, 5
2002, 5
2003, 9
2004, 5
2005, 3
2006, 4
2007, 7
2008, 7
2009, 7
2010, 7
2011, 6
2012, 8


#### Q7. (10 points) Write an SQL query to find all Sci-Fi movies from the U.S. ordered by increasing rating, then by decreasing year if ratings are the same.

```
Sci-Fi movies:Title, Year, Rating
District 9, 2009, 8.0
Star Trek, 2009, 8.0
The Truman Show, 1998, 8.0
Jurassic Park, 1993, 8.0
Twelve Monkeys, 1995, 8.1
Donnie Darko, 2001, 8.2
The Thing, 1982, 8.2
Blade Runner, 1982, 8.3
Eternal Sunshine of the Spotless Mind, 2004, 8.4
Star Wars: Episode VI - Return of the Jedi, 2002, 8.4
2001: A Space Odyssey, 1968, 8.4
WALL·E, 2008, 8.5
Aliens, 1986, 8.5
Back to the Future, 1985, 8.5
Alien, 1979, 8.5
Terminator 2: Judgment Day, 1991, 8.6
The Matrix, 1999, 8.7
Inception, 2010, 8.8
Star Wars: Episode V - The Empire Strikes Back, 2008, 8.8
Star Wars, 1977, 8.8
```

In [59]:
results = c.execute('SELECT Title, Year, Rating FROM movies WHERE imdb_id IN (SELECT imdb_id FROM movie_genre WHERE genre IS "Sci-Fi") ORDER BY rating, year DESC')
print("Sci-Fi movies:Title, Year, Rating") 
for row in results:
    print(row[0]+', '+repr(row[1])+', '+repr(row[2]))

Sci-Fi movies:Title, Year, Rating
District 9, 2009, 8.0
Star Trek, 2009, 8.0
The Truman Show, 1998, 8.0
Jurassic Park, 1993, 8.0
Twelve Monkeys, 1995, 8.1
The Terminator, 1984, 8.1
Stalker, 1979, 8.1
Donnie Darko, 2001, 8.2
The Thing, 1982, 8.2
Blade Runner, 1982, 8.3
Eternal Sunshine of the Spotless Mind, 2004, 8.4
Star Wars: Episode VI - Return of the Jedi, 2002, 8.4
2001: A Space Odyssey, 1968, 8.4
Metropolis, 1927, 8.4
WALL·E, 2008, 8.5
Aliens, 1986, 8.5
Back to the Future, 1985, 8.5
Alien, 1979, 8.5
A Clockwork Orange, 1971, 8.5
Terminator 2: Judgment Day, 1991, 8.6
The Matrix, 1999, 8.7
Inception, 2010, 8.8
Star Wars: Episode V - The Empire Strikes Back, 2008, 8.8
Star Wars, 1977, 8.8


#### Q8. (10 points) Write an SQL query to find the top 10 pickiest actors who played in the highest rated movies (on average) in and after year 2000. 
For each actor, give their name, average rating of the movies they played in, and the number of movies. Sort the result from pickest (highest rating) to the least picky. In case of ties, sort the rows by actor name.

```
In and after year 2000, top 10 pickiest actors who played in the highest rated movies: Actor, Movies
Aaron Eckhart, 1, 9.0
Anthony Michael Hall, 1, 9.0
Chin Han, 1, 9.0
Eric Roberts, 1, 9.0
Heath Ledger, 1, 9.0
Monique Gabriela Curnen, 1, 9.0
Nestor Carbonell, 1, 9.0
Ritchie Coster, 1, 9.0
Ron Dean, 1, 9.0
Alexandra Astin, 1, 8.9
```



In [60]:
results = c.execute('''SELECT actor, COUNT(*), Avg(rating)  FROM
(SELECT m_a.actor,m.rating FROM movie_actor AS m_a LEFT JOIN movies AS m ON m_a.imdb_id = m.imdb_id WHERE m.year >=2000 ORDER BY actor) 
GROUP BY actor ORDER BY Avg(rating) DESC LIMIT 10
''')
print("In and after year 2000, top 10 pickiest actors who played in the highest rated movies: Actor, Movies") 
for row in results:
    print(row[0]+', '+repr(row[1])+', '+repr(row[2]))

In and after year 2000, top 10 pickiest actors who played in the highest rated movies: Actor, Movies
Aaron Eckhart, 1, 9.0
Anthony Michael Hall, 1, 9.0
Chin Han, 1, 9.0
Eric Roberts, 1, 9.0
Heath Ledger, 1, 9.0
Monique Gabriela Curnen, 1, 9.0
Nestor Carbonell, 1, 9.0
Ritchie Coster, 1, 9.0
Ron Dean, 1, 9.0
Alexandra Astin, 1, 8.9


#### Q9. (10 points) Write an SQL query for finding pairs of actors who co-starred in at least 2 highly rated (rating > 9) movies together. 

The pairs of names must be unique. This means that ‘actor A, actor B’ and ‘actor B, actor A’ are the same pair, so only one of them should appear. For each pair of actors you print out, the two actors must be ordered alphabetically. The pairs are ordered in decreasing number of movies they co-stared in. In case of ties, the rows are ordered by actors’ names. 

>You will need to join the movie_actor table with itself to get this data (in addition to another required join). It is a bit tricky. If you cannot do it with SQL statement, you can also write some Python code that works on the Python data structure that you used to create the movie_actor table. That’ll mean much more lines of code, and if you do it that way, you’ll get 5 points instead of 10 points. You will only get 10 points if you solve it with pure SQL.

```
Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Al Pacino, Diane Keaton, 2
Al Pacino, John Cazale, 2
Al Pacino, Robert Duvall, 2
Al Pacino, Talia Shire, 2
Diane Keaton, John Cazale, 2
Diane Keaton, Robert Duvall, 2
Diane Keaton, Talia Shire, 2
John Cazale, Robert Duvall, 2
John Cazale, Talia Shire, 2
Robert Duvall, Talia Shire, 2
```

In [61]:
results = c.execute('''SELECT m_a.actor1,m_a.actor2,COUNT(*) FROM
(SELECT a.imdb_id, a.actor AS actor1, b.actor AS actor2 FROM movie_actor a, movie_actor b WHERE a.imdb_id = b.imdb_id AND a.actor < b.actor) AS m_a
LEFT JOIN movies AS m ON m_a.imdb_id = m.imdb_id WHERE m.rating > 9 GROUP BY actor1, actor2 HAVING COUNT(*) >= 2 ORDER BY m_a.actor1, m_a.actor2, COUNT(*) DESC
''')
print("Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies") 
for row in results:
    print(row[0]+', '+row[1]+', '+repr(row[2]))
conn.close()

Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Al Pacino, Diane Keaton, 2
Al Pacino, John Cazale, 2
Al Pacino, Robert Duvall, 2
Al Pacino, Talia Shire, 2
Diane Keaton, John Cazale, 2
Diane Keaton, Robert Duvall, 2
Diane Keaton, Talia Shire, 2
John Cazale, Robert Duvall, 2
John Cazale, Talia Shire, 2
Robert Duvall, Talia Shire, 2


## Part 2 (25 points) 

Write a python function that takes two arguments: genre and k 

The function should print out the top k actors who played roles in the highest rated movies (on average) in the provided genre. You should use the sqlite3 database file you created in Part 1. Some example runs of my function are shown below. Your program should produce the same output when provided with the same command line arguments.

`get_top_genre_actors("Drama", 5)`
```
Top 5 actors who played in highest rated Drama movies:
Actor, Average Drama movie rating
Bob Gunton, 9.3
Brian Libby, 9.3
Clancy Brown, 9.3
David Proval, 9.3
Gil Bellows, 9.3
```

`get_top_genre_actors("Comedy", 10)`
```
Top 10 actors who played in highest rated Comedy movies:
Actor, Average Comedy movie rating
Al Ernest Garcia, 8.6
Alba Gaïa Kraghede Bellugi, 8.6
Anne Le Ny, 8.6
Audrey Fleurot, 8.6
Cecil Reynolds, 8.6
Christian Ameri, 8.6
Cyril Mendy, 8.6
Dominique Daguier, 8.6
Dorothée Brière, 8.6
Edward LeSaint, 8.6
```


`get_top_genre_actors("Drama", 10)`
```
Top 10 actors who played in highest rated Drama movies:
Actor, Average Drama movie rating
Bob Gunton, 9.3
Brian Libby, 9.3
Clancy Brown, 9.3
David Proval, 9.3
Gil Bellows, 9.3
James Whitmore, 9.3
Joseph Ragno, 9.3
Larry Brandenburg, 9.3
Mark Rolston, 9.3
Neil Giuntoli, 9.3
```

In [62]:
def get_top_genre_actors(genre, number):
    conn = sqlite3.connect('theoliao_si618_hw2.db')
    c = conn.cursor()
    results = c.execute('''SELECT t.actor, AVG(m.rating) FROM
(SELECT imdb_id, actor FROM movie_actor WHERE imdb_id IN
(SELECT imdb_id FROM movie_genre WHERE genre IS ?)) AS t LEFT JOIN movies AS m ON t.imdb_id = m.imdb_id GROUP BY t.actor 
ORDER BY AVG(m.rating) DESC, t.actor
LIMIT ?''',(genre,number))
    print('Top {num} actors who played in highest rated {gen} movies:'.format(num = number,gen = genre)) 
    for row in results:
        print(row[0] +', '+ repr(row[1]))
    conn.close()

In [63]:
get_top_genre_actors("Drama", 5)

Top 5 actors who played in highest rated Drama movies:
Bob Gunton, 9.3
Brian Libby, 9.3
Clancy Brown, 9.3
David Proval, 9.3
Gil Bellows, 9.3


In [64]:
get_top_genre_actors("Comedy", 10)

Top 10 actors who played in highest rated Comedy movies:
Al Ernest Garcia, 8.6
Alba Gaïa Kraghede Bellugi, 8.6
Anne Le Ny, 8.6
Audrey Fleurot, 8.6
Cecil Reynolds, 8.6
Christian Ameri, 8.6
Cyril Mendy, 8.6
Dominique Daguier, 8.6
Dorothée Brière, 8.6
Edward LeSaint, 8.6


In [65]:
get_top_genre_actors("Drama", 10)

Top 10 actors who played in highest rated Drama movies:
Bob Gunton, 9.3
Brian Libby, 9.3
Clancy Brown, 9.3
David Proval, 9.3
Gil Bellows, 9.3
James Whitmore, 9.3
Joseph Ragno, 9.3
Larry Brandenburg, 9.3
Mark Rolston, 9.3
Neil Giuntoli, 9.3


## What to submit: 
Please perform the following operations and turn in the corresponding files to canvas:
1. Jupyter notebook titled uniqname_si618_hw2.ipynb and the corresponding html page (uniqname_si618_hw2.html) 
through Canvas.
2. The sqlite database file you created. (uniqname_si618_hw2.db)